In [24]:

import src
# TODO: Delete after development
import importlib
importlib.reload(src)

from src import Configuration, FileUtils, Analysis

In [25]:
authors = FileUtils.read_authors(Configuration.SELECTED_AUTHORS_FILEPATH)
authors

['Zane Grey',
 'Joseph Conrad',
 'Benjamin Disraeli',
 'Lucy Maud Montgomery',
 'William Henry Hudson']

In [26]:
authors_books = FileUtils.read_books(authors, Configuration.ALL_BOOK_DATA_DIR)

In [27]:
authors_generated_text = {}
for model, data_dir in Configuration.MODELS_DATA_DIR.items():
    authors_generated_text[model] = FileUtils.read_generated_texts(authors, data_dir)

In [30]:
analysis = {}
for model, generated_texts in authors_generated_text.items():
    analysis[model] = Analysis.analyse_generated_text(generated_texts, Configuration.SIZE_ANALYSIS)
analysis

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\piotr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


{'gemini-pro': {'Zane Grey': {'word_count': 5341,
   'unique_word_count': 2222,
   'average_word_length': 4.574237034263247,
   'average_sentence_length': 0.07302003370155402},
  'Joseph Conrad': {'word_count': 5179,
   'unique_word_count': 2176,
   'average_word_length': 4.758833751689515,
   'average_sentence_length': 0.05869859046147905},
  'Benjamin Disraeli': {'word_count': 4973,
   'unique_word_count': 2223,
   'average_word_length': 5.000402171727328,
   'average_sentence_length': 0.054695354916549366},
  'Lucy Maud Montgomery': {'word_count': 4998,
   'unique_word_count': 2260,
   'average_word_length': 4.96858743497399,
   'average_sentence_length': 0.05822328931572629},
  'William Henry Hudson': {'word_count': 5187,
   'unique_word_count': 2209,
   'average_word_length': 4.755542702911124,
   'average_sentence_length': 0.05668016194331984}},
 'gpt-4': {'Zane Grey': {'word_count': 5066,
   'unique_word_count': 2123,
   'average_word_length': 4.88768258981445,
   'average_sente

In [ ]:
# from functionwords import FunctionWords

# fw = FunctionWords(function_words_list="english")
# fw.transform(authors_gpt_3_5_turbo["Zane Grey"][0]["response"])
# fw
# # to list all function words given `function_words_list`
# # returns a list
# # fw.get_feature_names()